In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install tensorflow sklearn


In [4]:
def get_data_from_dir_to_dict(data_dir):
  '''
  This function take loction of data and return it in dict sepreated and return classes in list
  Args
    data_dir => location of the data
  
  return 
    classes => name of each class in List

  '''
  data_dir=pathlib.Path(data_dir)

  classes= [str(i).split('/')[-1].lower() for i in list(data_dir.glob('*'))]
  images={}
  for i in classes:
    images[i]=list(data_dir.glob(f'{i}/*'))

  return classes, images



In [5]:
data_dir='/content/drive/MyDrive/Vehicle Classification '

In [ ]:
# !fusermount -u drive
# !google-drive-ocamlfuse drive

/bin/bash: google-drive-ocamlfuse: command not found


In [6]:
# data_dir = pathlib.Path(data_dir)
# data_dir

classes,train_dir=get_data_from_dir_to_dict('/content/gdrive/MyDrive/Vehicle Classification /train')
print(classes)
classes,test_dir= get_data_from_dir_to_dict('/content/gdrive/MyDrive/Vehicle Classification /test')
print(classes)

# list(test_dir.glob('*'))
# classes= [str(i).split('/')[-1] for i in list(test_dir.glob('*'))]
# test_img={}
# for i in classes:
#   test_img[i]=list(test_dir.glob(f'{i}/*'))
# test_img.keys()

['bus', 'half cars', 'trella', 'mallaky', 'jampo', 'mini bus', 'mecropas']
['bus', 'half cars', 'mallaky', 'trella', 'jampo', 'mini bus', 'mecropas']


In [7]:
import cv2
X=[]
y=[]
none_img_type=0
for class_name, images in train_dir.items():
    print('class: DONE','  '*5, f' {class_name}')
    for image in images:
        n=[0 for _ in range(len(classes))]
        img = cv2.imread(str(image),0)
        
        if img is None:
            none_img_type+=1
        else:
            resized_img = cv2.resize(img,(128,128))
        # img = cv2.resize(img,(25,25))
        X.append(resized_img)
        n[classes.index(class_name)]=1
        y.append(n)

class: bus DONE
class: half cars DONE
class: trella DONE
class: mallaky DONE
class: jampo DONE
class: mini bus DONE
class: mecropas DONE


In [3]:
import pathlib
def convert_img_to_arr_with_class(img_dir_dict,num_classes,img_shape):
  X=[]
  y=[]
  none_img_type=0 
  for class_name, images in img_dir_dict.items():
    for image in images:
        n=[0 for _ in range(len(num_classes))]
        img = cv2.imread(str(image),0)
        
        if img is None:
            none_img_type+=1
        else:
            resized_img = cv2.resize(img,img_shape)
        # img = cv2.resize(img,(25,25))
        X.append(resized_img)
        n[classes.index(class_name)]=1
        y.append(n)
        if none_img_type >0:
          print()
        return (X,y) 


In [14]:
X1,y1=convert_img_to_arr_with_class(train_dir,classes,(128,128))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# train_dir=pathlib.Path('/content/gdrive/MyDrive/Vehicle Classification /train')
# test_dir= pathlib.Path('/content/gdrive/MyDrive/Vehicle Classification /test')

# train_data_img= ImageDataGenerator(1/255.)
# test_data_img= ImageDataGenerator(1/255.)
# x= train_data_img.flow_from_directory(train_dir,target_size=(128,128),color_mode='grayscale',class_mode='categorical',batch_size=42,)
# y= train_data_img.flow_from_directory(test_dir,target_size=(128,128),color_mode='grayscale',class_mode='categorical',batch_size=42,)



Found 2875 images belonging to 7 classes.
Found 2845 images belonging to 7 classes.


In [8]:
import numpy as np
X= np.array(X,dtype=np.float16)/255.
y= np.array(y)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.1,random_state=42)
X_train.shape,X_val.shape ,X_test.shape,y_train.shape,y_val.shape,y_test.shape

((1515, 128, 128),
 (169, 128, 128),
 (421, 128, 128),
 (1515, 7),
 (169, 7),
 (421, 7))

In [ ]:
len(y_train[0])

7

In [ ]:
import tensorflow as tf

cnn = tf.keras.Sequential([
      #  layer input
    layers.InputLayer(input_shape=(224, 224, 3)),

    # data augmentation
   tf.keras.layers.RandomContrast(0.2),
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomTranslation(0.2,0.1),
    layers.RandomZoom(0.3),
#     cnn
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.Conv2D(32,3,padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(16,3),
    tf.keras.layers.MaxPooling2D(),
    # NN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512,activation='relu',name='D1'),

tf.keras.layers.Dense(256, activation='relu',name='D2'),
tf.keras.layers.Dense(128, activation='relu',name='D3'), 
tf.keras.layers.Dense(64, activation='relu',name='D4'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(7,activation='softmax',name='D5'),

],name='Vehicle-Classifications')
cnn.compile(optimizer=tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy'])
cnn.summary()

Model: "Vehicle-Classifications"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 62, 62, 16)        4624      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 31, 31, 16)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 15376)             0         
                                                                 
 dropout_4 (Dropout)         (None, 15376) 

In [ ]:
history= cnn.fit(X,y,epochs=40,batch_size=42,
                 validation_data=(X_val,y_val)
                 )

Epoch 1/40
51/51 [==============================] - 10s 22ms/step - loss: 1.5558 - accuracy: 0.2665 - val_loss: 1.5022 - val_accuracy: 0.2663
Epoch 2/40
51/51 [==============================] - 1s 15ms/step - loss: 1.4963 - accuracy: 0.2874 - val_loss: 1.5014 - val_accuracy: 0.2722
Epoch 3/40
51/51 [==============================] - 1s 15ms/step - loss: 1.4680 - accuracy: 0.2855 - val_loss: 1.4659 - val_accuracy: 0.2840
Epoch 4/40
51/51 [==============================] - 1s 15ms/step - loss: 1.4654 - accuracy: 0.2770 - val_loss: 1.5645 - val_accuracy: 0.2722
Epoch 5/40
51/51 [==============================] - 1s 15ms/step - loss: 1.4370 - accuracy: 0.2903 - val_loss: 1.4753 - val_accuracy: 0.3609
Epoch 6/40
51/51 [==============================] - 1s 15ms/step - loss: 1.3095 - accuracy: 0.3919 - val_loss: 1.4022 - val_accuracy: 0.3846
Epoch 7/40
51/51 [==============================] - 1s 14ms/step - loss: 1.2235 - accuracy: 0.4717 - val_loss: 1.1728 - val_accuracy: 0.4970
Epoch 8/40
5

In [ ]:
cnn.evaluate(X_test,y_test)